# [LVV-T2213] - Look-up Table Application from MTMount Elevation Changes

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2213]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2213
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

In [ ]:
test_execution = "LVV-EXXXX" # Updated execution

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [ ]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst.ts import utils, salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

## (Not) Switching Components from Hardware to Simulator

We actually found out that `mtmount-sim` is publishing wrong elevation values.  
This causes M1M3 and M2 to go to a FAULT state because the angle between the inclinometer and the mount elevation is too large. 
Because of that, we are running this test using only `mtm2hex-sim` as a simulator.  
Everything else is running using hardware.

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Move mtmount to Zenith

Command the mount to elevation = 90, azimuth = 0, so that we can start m1m3 with LUT in mount telemetry mode).

In [ ]:
await mtcs.stop_tracking()

In [ ]:
# We are running this command because we do not want to track
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

---
## Get M1M3 Ready

### M1M3 LUT use mount telemetry

<p style="color: firebrick"><b> When M1M3 LUT is using the mount to get the elevation, avoid changes greater than 1 deg per command and avoid going lower than 82.5 deg </b></p>

1) Lower the mirror
2) Put M1M3 into the OFFLINE state
3) Access the m1m3-crio machine via SSH
4) Change the UseInclinometer parameter in the file below from True to False
   `/var/lib/ts-M1M3support/Sets/Default/1/ForceActuatorSettings.yaml`
5) Start M1M3 back again

In [ ]:
# If M1M3 is raised, make sure you lower it before setting the EUI/CSC to OFFLINE 
await mtcs.lower_m1m3()

In [ ]:
# If M1M3 was enabled before, disabled it first and enabled again to start fresh
await mtcs.set_state(salobj.State.OFFLINE, components=["mtm1m3"])

Access the `m1m3-crio` machine and edit the configuration file mentioned above to have M1M3 using the Mount Elevation instead of the Inclinometer. Restart the M1M3 CSC.

In [ ]:
# If M1M3 was enabled before, disabled it first and enabled again to start fresh
await mtcs.set_state(
    salobj.State.ENABLED, 
    components=["mtm1m3"],
    overrides=dict(mtm1m3="Default"),
)

In [ ]:
# Use this command to raise M1M3
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

## Get M2 Ready

- Need to have M2 LUT use mount telemetry - See [Use of M2 EUI on Summit]
- Turn on Force Balance system
- Clear forces

[Use of M2 EUI on Summit]: https://confluence.lsstcorp.org/display/LTS/Use+of+M2+EUI+on+Summit

In [ ]:
# Disable and Enable M2 so we can assure to start fresh
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])
await mtcs.set_state(salobj.State.ENABLED, components=["mtm2"])

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Disable and Enable CamHex so we can assure to start fresh
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_1"])
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_1"])

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compensation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Gather Data - Without AO

  * command the mount to elevation =86 deg, azimuth = 0
  * wait 39s
  * command the mount to elevation = 82 deg, azimuth = 0.

In [ ]:
# Set this to True when you actually want to run this test
t_start = time.Time.now()
t_start.format = "isot"
print(f"Gathering data - without AO - Start time: {t_start.utc}")

In [ ]:
# await asyncio.sleep(39.)

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0., elevation=89.5)

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0., elevation=89.)

In [ ]:
time_sleep = 1.
step_size = 0.25

# Safely slew from 90. to 86.5 so M1M3 and M2 do not go to a fault
for el in np.arange(90., 86.4, -step_size):
    print(f"Moving elevation to {el:.2f} deg")
    await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0., elevation=el)
    await asyncio.sleep(time_sleep)

In [ ]:
# Slew to 86 deg
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0., elevation=86.)

# Check that we are actually tracking
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

await asyncio.sleep(39.)

await vandv.m1m3.plotM1M3Forces(mtcs.rem.mtm1m3)

axialForces = await mtcs.rem.mtm2.tel_axialForce.aget(timeout=2)
tangentForces = await mtcs.rem.mtm2.tel_tangentForce.aget(timeout=2)
vandv.m2.plotM2Forces(axialForces, tangentForces)

a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=1)
elev = a.elevation

await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_1)    
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=2)
elev = a.elevation

await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_2)    
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Safely slew from 86. to 83. so M1M3 and M2 do not go to a fault
for el in [85., 84.]:
    await mtcs.point_azel(az=0, el=el)
    await asyncio.sleep(1)

# Slew to 83 deg
await mtcs.point_azel(az=0, el=83)
await asyncio.sleep(39.)

t_end = time.Time.now()
t_end.format = "isot"
print(f"Gathering data - without AO - End time: {t_end.utc}")

await mtcs.stop_tracking()

## Plot Data

In [ ]:
# t_start = "2022-06-06T16:33:47.387"
# t_start = time.Time(t_start, format="isot", scale="tai")

# t_end = "2022-06-06T16:36:25.569"
# t_end = time.Time(t_end, format="isot", scale="tai")

## Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
t_start.utc

In [ ]:
df = await get_data_from_efd(
    exec_info.loc, 
    t_start, 
    t_end)

In [ ]:
df["elevation"].dropna()

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=3, sharex=True)

axs[0].plot(df["elevation"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["mtm2.axialForce.applied0"].dropna(), "C1^-", label="applied")
axs[1].plot(df["mtm2.axialForce.lutGravity0"].dropna(), "C2v-", label="Gravity LUT")
axs[1].set_ylabel("M2 Forces\n[--]")

axs[2].plot(df["mthexapod_1.application.position1"].dropna(), "C3x-", label="CamHex Y")
axs[2].plot(df["mthexapod_2.application.position1"].dropna(), "C4+-", label="M2Hex Y")
axs[2].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time.png")
plt.show()

In [ ]:
vandv.m1m3.plot_m1m3_and_elevation(df)a

In [ ]:
client = EfdClient("summit_efd")

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', t_start, t_end, 1)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', t_start, t_end, 1)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', t_start, t_end, 1)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', t_start, t_end, 1)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition, '--', label='mount elevation')
# plt.plot(dfm1m3.yForces0, label='M1M3 elevation y-force 101')
# plt.plot(dfm2.lutGravity0, label='M2 elevation force B1')
# plt.plot(dfh1.position1, label='Camera hexapod y')
# plt.plot(dfh2.position1, label='M2 hexapod y')
plt.grid()
plt.legend()

## M1M3 Elevation Forces vs LUT

Check the M1M3 elevation forces match what we expect from the implemented LUT.

In [ ]:
elevation = await mtcs.rem.mtmount.tel_elevation.aget(timeout=10.)
print(elevation, "\n")

The look-up table for X has basically zeroes.  
At least for now.  
So it does not make much sense evaluating it now.

The lut for Y has non-zeroes and zeroes values.  
in order to compare, we can drop the zeroes components.

For z, we are good. We can perform a direct comparison.

In [ ]:
lut_el_xforces = vandv.m1m3.lut_elevation_xforces(elevation.actualPosition)
lut_el_yforces = vandv.m1m3.lut_elevation_yforces(elevation.actualPosition)
lut_el_zforces = vandv.m1m3.lut_elevation_zforces(elevation.actualPosition)

In [ ]:
fel = await mtcs.rem.mtm1m3.evt_appliedElevationForces.aget(timeout=10.)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

ax.plot(fel.xForces, "C0^-", label="Applied")
ax.set_ylabel("Elevation xForces [??]")
ax.grid(":", alpha=0.2)
ax.legend()

fig.suptitle(f"{test_execution} - M1M3 Elevation Forces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_xForces.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 6), nrows=2, sharex=True)

axs[0].plot(fel.yForces, "C0^-", label="Applied")
axs[0].plot(lut_el_yforces[lut_el_yforces != 0], "C1v-", label="LUT")
axs[0].set_ylabel("Elevation yForces [??]")
axs[0].grid(":", alpha=0.2)
axs[0].legend()

axs[1].plot(fel.yForces - lut_el_yforces[lut_el_yforces != 0], label="Applied - LUT forces")
axs[1].set_ylabel("Elefation yForces \n difference [??]")
axs[1].grid(":", alpha=0.2)

fig.suptitle(f"{test_execution} - M1M3 Elevation yForces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_yForces.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 6), nrows=2, sharex=True)

axs[0].plot(fel.zForces, "C0^-", label="Applied")
axs[0].plot(lut_el_zforces, "C1v-", label="LUT")
axs[0].set_ylabel("Elevation zForces [??]")
axs[0].grid(":", alpha=0.2)
axs[0].legend()

axs[1].plot(fel.zForces - lut_el_zforces, label="Applied - LUT forces")
axs[1].set_ylabel("Elefation zForces \n difference [??]")
axs[1].grid(":", alpha=0.2)

fig.suptitle(f"{test_execution} - M1M3 Elevation zForces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_zForces.png")
plt.show()

The following plots are extracted from Bo's Notebooks.  
The originals are in [lsst-ts/ts_notebooks/bxin/ptg2m1m3].

[lsst-ts/ts_notebooks/bxin/ptg2m1m3]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/ptg2m1m3/m1m3_diagnostic.ipynb

## M2 Elevation Forces vs LUT

Check the M2 elevation forces match what we expect from the implemented LUT.

In [ ]:
vandv.m2.plot_m2_actuators()

In [ ]:
axialForces = await mtcs.rem.mtm2.tel_axialForce.aget(timeout=2)
tangentForces = await mtcs.rem.mtm2.tel_tangentForce.aget(timeout=2)

In [ ]:
vandv.m2.plotM2Forces(axialForces, tangentForces)

## CamHex Vs LUT

Check the camera hexapod LUT compensations match what we expect from the implemented LUT

In [ ]:
a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=1)
elev = a.elevation

await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_1)    
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

## M2Hex vs LUT

In [ ]:
a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=2)
elev = a.elevation

await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_2)    
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

### Close up

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
target = mtcs.radec_from_azel(az=0, el=80)

await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)
await mtcs.stop_tracking()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )

## Wrap Up

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3", "mtm2", "mthexapod_1", "mthexapod_2"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm2", "mthexapod_1", "mthexapod_2"])